**ULUROVER_SoilAnalysis_1**

***Importing Required Libraries***

In [ ]:
import os
import re
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from random import randrange
import matplotlib.pyplot as plt
from tensorflow.python.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

***Loading Data***

In [ ]:
train_data_dir = 'D:/Users/kayai/Downloads/ULUROVER_SoilAnalysis_SoilTypes/train'
test_data_dir = 'D:/Users/kayai/Downloads/ULUROVER_SoilAnalysis_SoilTypes/test'

In [ ]:
img_height = 224
img_width = 223

batch_size = 32

***Model Building***

*Choosing Model Parameters*

In [ ]:
num_conv_layers = 3
num_dense_layers = 2
layer_size = 28 # Limitting layer size to prevent overfitting
num_training_epochs = 5
MODEL_NAME = 'soil'

CLASSES = ['clay', 'gravel', 'humus', 'sand', 'silt']

***Defining Model Architecture***

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_dense_layers, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

***Preprocessing Data***

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    zoom_range = 0.2,
    rotation_range = 10,
    shear_range = 0.2,
    height_shift_range = 0.1,
    width_shift_range = 0.1
)

In [ ]:
# it is not necessary to create multiple variations for the testing images
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'sparse'
)

Found 15 images belonging to 5 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'sparse',
    shuffle = False
)

Found 15 images belonging to 5 classes.


***Model Training***

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_training_epochs
)

***Model Evaluation***

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nTest Accuracy: ', test_acc)

***Saving Trained Model***

In [ ]:
model.save('ULUROVER_Soil_Type_Classifier.h5')